# Pandas in velepodatki

V tej enoti se boste seznanili z uporabo knjižnice Pandas na velikih količinah podatkov. Kot velepodatke smatramo podatke, ki lahko dosegajo velikosti več TB ali pa celo še več. Velepodatki obsegajo na milijone, milijarde ali več vrstic v datotekah CSV ali JSON. V teh primerih se ukvarjamo z izzivi hitrega nalaganja, branja, filtriranja, obdelovanja in shranjevanja podatkov. Ob tako velikih količinah podatkov je zelo pomembno, da uporabimo smiselne pristope za delo s podatki, saj nam to lahko prihrani ogromno časa pri obdelavi. V nadaljevanju bomo podrobneje predstavili delo z velepodatki in knjižnico Pandas.

## Podatkovna zbirka KASANDR

Pri nadaljnjem delu bomo uporabljali podatkovno zbirko KASANDR, ki je prostodostopna zbirka velepodatkov. Zajema nekaj čez 17 milijonov vrstic podatkov o ogledih artiklov na spletnih straneh Kelkoo.com. Podatkovna zbirka KASANDR je bila zasnovana za razvoj priporočilnih sistemov.

Podatkovno zbirko najdemo v repozitoriju UCI Machine Learning (https://archive.ics.uci.edu/dataset/385/kasandr). Razdeljena je na testno in učno množico. Testna množica obsega skoraj 2 milijona vrstic in zaseda 381,3 MB, učna množica pa obsega skoraj 16 milijonov vrstic in zaseda 3,1 GB.

S spodnjo kodo prenesemo podatkovno zbirko in jo razširimo na sistem. **Kodo je v nadaljevanju priporočljivo zaganjati v okolju [Google Colab](https://colab.research.google.com).**

In [1]:
# Prenos podatkovne zbirke KASANDR (stisnjena: ~900MB, razširjena: 3.5GB; 17.764.280 vrstic, 7 stolpcev)
# https://archive.ics.uci.edu/static/public/385/kasandr.zip

print('Prenos podatkovne zbirke KASANDR ...')
!curl https://archive.ics.uci.edu/static/public/385/kasandr.zip -o kasandr.zip
print('Prenos končan!')

print('Razširjanje podatkovne zbirke KASANDR (korak 1/2) ...')
!unzip kasandr.zip
print('Razširjanje podatkovne zbirke KASANDR (korak 2/2) ...')
!tar -xvjf de.tar.bz2
print('Razširjanje končano!')

# počistimo začasne datoteke
!rm -rf de.tar* kasandr.zip

Prenos podatkovne zbirke KASANDR ...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  900M    0  900M    0     0  2906k      0 --:--:--  0:05:17 --:--:-- 11.3M
Prenos končan!
Razširjanje podatkovne zbirke KASANDR (korak 1/2) ...
Archive:  kasandr.zip
 extracting: de.tar.bz2              
Razširjanje podatkovne zbirke KASANDR (korak 2/2) ...
./._de
de/
de/._test_de.csv
de/test_de.csv
de/._train_de.csv
de/train_de.csv
Razširjanje končano!


In [2]:
# vključevanje potrebnih knjižnic
import os
import humanize
import pandas as pd

In [3]:
# preverimo velikost datotek
size_test = os.path.getsize('de/test_de.csv')
size_train = os.path.getsize('de/train_de.csv')

print(f'Velikosti datotek podatkovne zbirke KASANDR: \n- test_de.csv: {humanize.naturalsize(size_test)} \n- train_de.csv: {humanize.naturalsize(size_train)}')

Velikosti datotek podatkovne zbirke KASANDR: 
- test_de.csv: 381.3 MB 
- train_de.csv: 3.1 GB


In [4]:
# poskusimo prebrati datoteko test_de.csv
# s posebnim ukazom %time izmerimo čas izvajanja
# v read_csv() uporabimo separator '\t', saj so podatki ločeni s tabulatorjem
%time df = pd.read_csv('de/test_de.csv', sep='\t')

CPU times: user 5.69 s, sys: 787 ms, total: 6.48 s
Wall time: 6.49 s


In [5]:
# poglejmo še informacije o naloženi datoteki
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1919561 entries, 0 to 1919560
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   userid       object
 1   offerid      object
 2   countrycode  object
 3   category     int64 
 4   merchant     object
 5   utcdate      object
 6   rating       int64 
dtypes: int64(2), object(5)
memory usage: 886.0 MB


Nalaganje datoteke ``test_de.csv`` je trajalo okoli 10 sekund in pri tem zavzelo 886,0 MB pomnilnika. Poskusimo naložiti datoteko ``train_de.csv``:

In [6]:
%time df = pd.read_csv('de/train_de.csv', sep='\t')

CPU times: user 1min 7s, sys: 9.4 s, total: 1min 16s
Wall time: 1min 18s


In [7]:
# še enkrat poglejmo informacije o naloženi datoteki (train_de.csv)
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15844717 entries, 0 to 15844716
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   userid       object
 1   offerid      object
 2   countrycode  object
 3   category     int64 
 4   merchant     object
 5   utcdate      object
 6   rating       int64 
dtypes: int64(2), object(5)
memory usage: 7.1 GB


Tokrat je nalaganje datoteke trajalo približno 1 minuto in pol, pri tem pa zavzelo 7.1 GB pomnilnika. Jasno vidimo problem velepodatkov - enostavno jih je preveč, da bi lahko obdelali vse naenkrat. Potrebujemo torej način za postopno nalaganje podatkovne zbirke, kjer jo beremo po kosih.

## Postopno nalaganje podatkov

V Pandas lahko podatke naložimo postopoma po kosih. To dosežemo tako, da v funkcijo ``read_csv()`` podamo parameter ``chunksize``. Število, ki ga podamo kot vrednost, bo določalo koliko vrstic bomo prebrali naenkrat. Poskusimo na datoteki ``train_de.csv``:

In [8]:
# preberimo datoteko train_de.csv po delih (chunksize=50000)
%time df = pd.read_csv('de/train_de.csv', sep='\t', chunksize=50000)

CPU times: user 714 µs, sys: 968 µs, total: 1.68 ms
Wall time: 1.69 ms


Tokrat smo prebrali podatke v nekaj milisekundah, kar je bistvena pohitritev.

Če uporabimo parameter ``chunksize``, potem rezultat ni več Pandas DataFrame, temveč iterator. Če želimo dostopati do prebranih podatkov, potem moramo to tudi upoštevati. Poglejmo si kako dostopamo do podatkov, ki so bili prebrani po kosih:

In [9]:
# preberemo datoteko po kosih
iterator = pd.read_csv('de/train_de.csv', sep='\t', chunksize=50000)

for idx, chunk in enumerate(iterator):
  if(idx == 0): # izpišemo samo prvi kos (50000 vrstic)
    print(chunk) # spremenljivka chunk je DataFrame s podatki (50000 vrstic)
  else:
    break

                                                  userid  \
0      fa937b779184527f12e2d71c711e6411236d1ab59f8597...   
1      f6c8958b9bc2d6033ff4c1cc0a03e9ab96df4bcc528913...   
2      02fe7ccf1de19a387afc8a11d08852ffd2b4dabaed4e2d...   
3      9de5c06d0a16256b13b8e7cdc50bf203ecef533eb5cbe1...   
4      8d26ade603ea5473c3844aebfcd9e96e6adc8ff411576e...   
...                                                  ...   
49995  4ce7facb26d9f9af69e545f6475c63d3c10f990c743499...   
49996  0e07f12aec4a43388d70a8cfb8f242faf93478c782307b...   
49997  3c37546198bdee4f4daa0103d225568c2d3d5e0fd33a70...   
49998  1f996ba4c6b01f5bedb0d31f24e2148ff3f3b5ca5d4233...   
49999  1f996ba4c6b01f5bedb0d31f24e2148ff3f3b5ca5d4233...   

                                offerid countrycode   category  \
0      c5f63750c2b5b0166e55511ee878b7a3          de  100020213   
1      19754ec121b3a99fff3967646942de67          de  100020213   
2      5ac4398e4d8ad4167a57b43e9c724b18          de     125801   
3      be83df97

Z nekaj spremembami zgornje kode lahko dosežemo postopno nalaganje podatkov, ki jih potem tudi obdelamo. Ideja je, da naložimo manjši kos podatkovne zbirke, ga ustrezno obdelamo in shranimo. Če to ponovimo nad vsemi kosi in jih v istem vrstnem redu shranjujemo v novo datoteko, smo uspešno obdelali celotno podatkovno zbirko velepodatkov.

## Dobre prakse in namigi

Kadar se ukvarjamo z velepodatki, si jih želimo optimalno obdelati. Zaradi ogromne velikosti datotek in količine podatkov, je to včasih težko. V nadaljevanju je navedenih nekaj dobrih praks in namigov za delo z velepodatki.

### Predhodno filtriranje velepodatkov

Če poznamo strukturo velepodatkov in vemo, da nekaterih stolpcev ne bomo obdelovali, jih lahko pri nalaganju izpustimo. To storimo s funkcijo ``read_csv()`` in parametrom ``usecols``. Kot primer vzemimo stolpce v podatkovni zbirki KASANDR. Če bi nas zanimali zgolj identifikatorji uporabnikov, trgovcev in ocena, bi podatkovno zbirko naložili s spodnjo kodo:

In [10]:
%time df = pd.read_csv('de/train_de.csv', sep='\t', usecols=['userid', 'merchant', 'rating'])

CPU times: user 38.4 s, sys: 2.01 s, total: 40.5 s
Wall time: 41.1 s


V tem primeru smo vseh ~17 milijon vrstic naložili v približno 40 sekundah in pri tem porabili 3.7 GB pomnilnika. To je približno pol manj porabljenega časa in pol manj porabljenega pomnilnika! Vidimo, da je predhodno filtriranje lahko zelo koristno, saj enostavno ohranimo zgolj tiste podatke, katere bomo zagotovo obdelovali.

#### Kaj pa, če ne poznamo strukture velepodatkov?

V primerih, kadar ne poznamo strukture velepodatkov, predhodnega filtriranja velepodatkov ne moremo izvesti takoj, saj ne moremo podati vrednosti za parameter ``usecols``. Kako torej ugotovimo strukturo velepodatkov v datoteki CSV? Če imamo srečo, je struktura podana kot informacija ob datoteki CSV, sicer pa jo lahko z nekaj triki poskusimo ugotoviti.

CSV datoteke imajo glavo *(ang. header)*, ki hrani imena stolpcev. Izkaže se, da lahko iz CSV datoteke preberemo zgolj glavo in s tem pridobimo strukturo velepodatkov. To lahko storimo s knjižnico ``csv``, ki je vgrajena v Python. Uporabimo spodnjo kodo:

In [11]:
import csv

%time reader = csv.DictReader(open('de/train_de.csv', 'r'), delimiter='\t')

print(f'Struktura podatkovne zbirke KASANDR: {reader.fieldnames}')

CPU times: user 211 µs, sys: 0 ns, total: 211 µs
Wall time: 219 µs
Struktura podatkovne zbirke KASANDR: ['userid', 'offerid', 'countrycode', 'category', 'merchant', 'utcdate', 'rating']


Vse skupaj traja manj kot milisekundo, saj ``DictReader`` ne prebere datoteke v celoti. Imena stolpcev najdemo shranjene pod lastnostjo ``fieldnames``.

**Opomba:** V tem primeru smo vedeli, da so posamezni atributi v naši datoteki CSV ločeni s tabulatorjem (`` \t ``). Privzeto je za CSV datoteke znak za ločevanje vejica (``,``). Če res ne vemo nič o podatkovni zbirki, potem nam na tem mestu preostane le, da poskušamo z različnimi ločilnimi znaki.

Podobno funkcionalnost nam omogoča tudi knjižnica Pandas. Poglejmo na primeru:

In [12]:
%time cols = pd.read_csv('de/train_de.csv', nrows=0, sep='\t').columns.tolist()

print(f'Struktura podatkovne zbirke KASANDR: {cols}')

CPU times: user 6 ms, sys: 0 ns, total: 6 ms
Wall time: 6.21 ms
Struktura podatkovne zbirke KASANDR: ['userid', 'offerid', 'countrycode', 'category', 'merchant', 'utcdate', 'rating']


V zgornji kodi smo podali parameter ``nrows``, ki določa koliko vrstic s podatki bomo prebrali. Vrednost smo nastavili na 0, saj ne želimo brati podatkov, temveč samo glavo. Dobljen rezultat je podatkovna struktura DataFrame, ki ima lastnost ``columns`` in hrani imena stolpcev. Rezultat s funkcijo ``tolist()`` pretvorimo v seznam. Ta rešitev nam vrne rezultat v približno 10 milisekundah, kar je počasneje kot rešitev z uporabo vgrajene knjižnice ``csv``.

### Uporaba optimalnih podatkovnih tipov

V določenih primerih lahko delo s podatki olajšamo in pohitrimo z uporabo optimalnih podatkovnih tipov. Poglejmo porabo pomnilnika za podatkovne tipe na primeru predhodno filtrirane podatkovne zbirke KASANDR, ki porabi 3,7 GB pomnilnika:

In [13]:
# informacije o predhodno filtrirani podatkovni zbirki KASANDR
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15844717 entries, 0 to 15844716
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   userid    object
 1   merchant  object
 2   rating    int64 
dtypes: int64(1), object(2)
memory usage: 3.7 GB


In [14]:
# izpis podatkovnih tipov
df.dtypes

userid      object
merchant    object
rating       int64
dtype: object

In [15]:
# poraba pomnilnika po podatkovnih tipih
df.memory_usage(deep=True)

Index              128
userid      1917210757
merchant    1917210757
rating       126757736
dtype: int64

In [16]:
# podrobneje poglejmo vrednosti stolpca 'rating'
df.value_counts('rating')

rating
0    15139270
1      705447
Name: count, dtype: int64

Iz rezultatov zgornjih treh ukazov vidimo, da stolpca ``userid`` in ``merchant`` (podatkovni tip ``object``) vzameta največ pomnilnika. Stolpec ``rating`` je predstavljen s podatkovnim tipom ``int64``, čeprav se dejansko uporabljata zgolj vrednosti 0 in 1. Stolpca ``userid`` in ``merchant`` sta dobra kandidata za pretvorbo v kategorični tip, stolpec ``rating`` za pretvorbo v bolj primeren podatkovni tip (npr. ``bool``). Naredimo kopijo podatkovne zbirke v pomnilniku in ji določimo optimalne podatkovne tipe:

In [17]:
# kopija podatkovne zbirke v pomnilniku
df2 = df.copy()

# stolpca 'userid' in 'merchant' pretvorimo v tip 'category'
df2['userid'] = df2['userid'].astype('category')
df2['merchant'] = df2['merchant'].astype('category')

# stolpec 'rating' pretvorimo v tip 'bool'
df2['rating'] = df2['rating'].astype('bool')

# ponovno preverimo porabo pomnilnika
df2.memory_usage(deep=True)

Index             128
userid      107102737
merchant     31791049
rating       15844717
dtype: int64

Po pretvorbi v optimalne podatkovne tipe vidimo, da je poraba pomnilnika bistveno manjša. Natančneje preverimo koliko manjša je poraba pomnilnika:

In [18]:
df2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15844717 entries, 0 to 15844716
Data columns (total 3 columns):
 #   Column    Dtype   
---  ------    -----   
 0   userid    category
 1   merchant  category
 2   rating    bool    
dtypes: bool(1), category(2)
memory usage: 147.6 MB


In [19]:
diff = df2.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()

print(f'Izboljšava: {diff:0.2f}')

Izboljšava: 0.04


Podatkovna zbirka z optimalnimi podatkovnimi tipi zdaj zaseda le 147,6 MB. Podatkovna zbirka z optimalnimi podatkovnimi tipi (``df2``) porabi **4 %** pomnilnika, ki ga porabi predhodno filtrirana podatkovna zbirka (``df``). Zaključimo lahko, da smo drastično pomanjšali pomnilniške zahteve naših velepodatkov z upoštevanjem optimalnih podatkovnih tipov.

**Opomba:** alternativno lahko prepustimo knjižnici Pandas, da samodejno optimalno izbere numerične tipe. Poglejmo na primeru:

In [20]:
# uporaba funkcije to_numeric() za avtomatsko optimalno pretvorbo podatkovnega tipa; rezultat bo tipa 'bool'
df2['rating'] = pd.to_numeric(df2['rating'], downcast='unsigned')

### Dodatni viri s primeri

[Dokumentacija Pandas](https://pandas.pydata.org/docs/user_guide) ima odlične vodiče, katere vam priporočamo, da jih pregledate in preizkusite. Za delo z velepodatki so najbolj relevantni naslednji vodiči, ki jih lahko uporabite kot dodatno gradivo:

* [Scaling to large datasets](https://pandas.pydata.org/docs/user_guide/scale.html)
* [Sparse data structures](https://pandas.pydata.org/docs/user_guide/sparse.html)
* [IO tools (text, CSV, HDF5, ...)](https://pandas.pydata.org/docs/user_guide/io.html)
* [Enhancing performance](https://pandas.pydata.org/docs/user_guide/enhancingperf.html)